In [1]:
import asyncio
import time
from indy import anoncreds, crypto, did, ledger, pool, wallet, blob_storage

import json
from typing import Optional
import os

async def onboarding(_from, to):
    print("\"{}\" -> Create and store in Wallet \"{} {}\" DID".format(_from['name'], _from['name'], to['name']))
    (from_to_did, from_to_key) = await did.create_and_store_my_did(_from['wallet'], "{}")

    print("\"{}\" -> Send Nym to Ledger for \"{} {}\" DID".format(_from['name'], _from['name'], to['name']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], from_to_did, from_to_key, None)

    print("\"{}\" -> Send connection request to {} with \"{} {}\" DID and nonce"
                .format(_from['name'], to['name'], _from['name'], to['name']))
    connection_request = {
        'did': from_to_did,
        'nonce': 123456789
    }

    if 'wallet' not in to:
        print("\"{}\" -> Create wallet".format(to['name']))
        try:
            await wallet.create_wallet(to['wallet_config'], to['wallet_credentials'])
        except IndyError as ex:
            if ex.error_code == ErrorCode.PoolLedgerConfigAlreadyExistsError:
                pass
        to['wallet'] = await wallet.open_wallet(to['wallet_config'], to['wallet_credentials'])

    print("\"{}\" -> Create and store in Wallet \"{} {}\" DID".format(to['name'], to['name'], _from['name']))
    (to_from_did, to_from_key) = await did.create_and_store_my_did(to['wallet'], "{}")

    print("\"{}\" -> Get key for did from \"{}\" connection request".format(to['name'], _from['name']))
    from_to_verkey = await did.key_for_did(_from['pool'], to['wallet'], connection_request['did'])

    print("\"{}\" -> Anoncrypt connection response for \"{}\" with \"{} {}\" DID, verkey and nonce"
                .format(to['name'], _from['name'], to['name'], _from['name']))
    to['connection_response'] = json.dumps({
        'did': to_from_did,
        'verkey': to_from_key,
        'nonce': connection_request['nonce']
    })
    to['anoncrypted_connection_response'] = \
        await crypto.anon_crypt(from_to_verkey, to['connection_response'].encode('utf-8'))

    print("\"{}\" -> Send anoncrypted connection response to \"{}\"".format(to['name'], _from['name']))
    _from['anoncrypted_connection_response'] = to['anoncrypted_connection_response']

    print("\"{}\" -> Anondecrypt connection response from \"{}\"".format(_from['name'], to['name']))
    _from['connection_response'] = \
        json.loads((await crypto.anon_decrypt(_from['wallet'], from_to_key,
                                              _from['anoncrypted_connection_response'])).decode("utf-8"))

    print("\"{}\" -> Authenticates \"{}\" by comparision of Nonce".format(_from['name'], to['name']))
    assert connection_request['nonce'] == _from['connection_response']['nonce']

    print("\"{}\" -> Send Nym to Ledger for \"{} {}\" DID".format(_from['name'], to['name'], _from['name']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], to_from_did, to_from_key, None)

    return from_to_did, from_to_key, to_from_did, to_from_key, _from['connection_response']


async def get_verinym(_from, from_to_did, from_to_key, to, to_from_did, to_from_key):
    print("\"{}\" -> Create and store in Wallet \"{}\" new DID".format(to['name'], to['name']))
    (to_did, to_key) = await did.create_and_store_my_did(to['wallet'], "{}")

    print("\"{}\" -> Authcrypt \"{} DID info\" for \"{}\"".format(to['name'], to['name'], _from['name']))
    to['did_info'] = json.dumps({
        'did': to_did,
        'verkey': to_key
    })
    to['authcrypted_did_info'] = \
        await crypto.auth_crypt(to['wallet'], to_from_key, from_to_key, to['did_info'].encode('utf-8'))

    print("\"{}\" -> Send authcrypted \"{} DID info\" to {}".format(to['name'], to['name'], _from['name']))

    print("\"{}\" -> Authdecrypted \"{} DID info\" from {}".format(_from['name'], to['name'], to['name']))
    sender_verkey, authdecrypted_did_info_json, authdecrypted_did_info = \
        await auth_decrypt(_from['wallet'], from_to_key, to['authcrypted_did_info'])

    print("\"{}\" -> Authenticate {} by comparision of Verkeys".format(_from['name'], to['name'], ))
    assert sender_verkey == await did.key_for_did(_from['pool'], _from['wallet'], to_from_did)

    print("\"{}\" -> Send Nym to Ledger for \"{} DID\" with {} Role"
                .format(_from['name'], to['name'], to['role']))
    await send_nym(_from['pool'], _from['wallet'], _from['did'], authdecrypted_did_info['did'],
                   authdecrypted_did_info['verkey'], to['role'])

    return to_did


async def send_nym(pool_handle, wallet_handle, _did, new_did, new_key, role):
    nym_request = await ledger.build_nym_request(_did, new_did, new_key, None, role)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, nym_request)


async def send_schema(pool_handle, wallet_handle, _did, schema):
    schema_request = await ledger.build_schema_request(_did, schema)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, schema_request)


async def send_cred_def(pool_handle, wallet_handle, _did, cred_def_json):
    cred_def_request = await ledger.build_cred_def_request(_did, cred_def_json)
    await ledger.sign_and_submit_request(pool_handle, wallet_handle, _did, cred_def_request)


async def get_schema(pool_handle, _did, schema_id):
    get_schema_request = await ledger.build_get_schema_request(_did, schema_id)
    get_schema_response = await ledger.submit_request(pool_handle, get_schema_request)
    return await ledger.parse_get_schema_response(get_schema_response)


async def get_cred_def(pool_handle, _did, cred_def_id):
    get_cred_def_request = await ledger.build_get_cred_def_request(_did, cred_def_id)
    get_cred_def_response = await ledger.submit_request(pool_handle, get_cred_def_request)
    return await ledger.parse_get_cred_def_response(get_cred_def_response)


async def get_credential_for_referent(search_handle, referent):
    credentials = json.loads(
        await anoncreds.prover_fetch_credentials_for_proof_req(search_handle, referent, 10))
    return credentials[0]['cred_info']


async def prover_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_states = {}
    for item in identifiers.values():
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Create Revocation States

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_states)


async def verifier_get_entities_from_ledger(pool_handle, _did, identifiers, actor):
    schemas = {}
    cred_defs = {}
    rev_reg_defs = {}
    rev_regs = {}
    for item in identifiers:
        print("\"{}\" -> Get Schema from Ledger".format(actor))
        (received_schema_id, received_schema) = await get_schema(pool_handle, _did, item['schema_id'])
        schemas[received_schema_id] = json.loads(received_schema)

        print("\"{}\" -> Get Credential Definition from Ledger".format(actor))
        (received_cred_def_id, received_cred_def) = await get_cred_def(pool_handle, _did, item['cred_def_id'])
        cred_defs[received_cred_def_id] = json.loads(received_cred_def)

        if 'rev_reg_seq_no' in item:
            pass  # TODO Get Revocation Definitions and Revocation Registries

    return json.dumps(schemas), json.dumps(cred_defs), json.dumps(rev_reg_defs), json.dumps(rev_regs)


async def auth_decrypt(wallet_handle, key, message):
    from_verkey, decrypted_message_json = await crypto.auth_decrypt(wallet_handle, key, message)
    decrypted_message_json = decrypted_message_json.decode("utf-8")
    decrypted_message = json.loads(decrypted_message_json)
    return from_verkey, decrypted_message_json, decrypted_message

In [2]:
import asyncio
import time
from indy import anoncreds, crypto, did, ledger, pool, wallet, blob_storage
import os
import json
from typing import Optional
from ipywidgets import Box, Layout, VBox, Output, HTML
import ipywidgets as widgets

async def run():
    print("Getting started -> started")
    print("=== 1/9: Open Ledger  ==")


    # Set protocol version 2 to work with Indy Node 1.4
    await pool.set_protocol_version(2)
    
    pool_ = {
        'name': 'pool1',
        'config': json.dumps({"genesis_txn": '/home/indy/sandbox/pool_transactions_genesis'})
    }
    print("Open Pool Ledger: {}".format(pool_['name']))

    try:
        await pool.create_pool_ledger_config(pool_['name'], pool_['config'])
    except:
        pass
    pool_['handle'] = await pool.open_pool_ledger(pool_['name'], None)

    print("==============================")
    print("== 2/9: Setup Steward  ==")
    print("=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==")
    print("------------------------------")

    print("\"Sovrin Steward\" -> Create wallet")
    steward = {
        'name': "Sovrin Steward",
        'wallet_config': json.dumps({'id': 'sovrin_steward_wallet'}),
        'wallet_credentials': json.dumps({'key': 'steward_wallet_key'}),
        'pool': pool_['handle'],
        'seed': '000000000000000000000000Steward1'
    }

    try:
        await wallet.create_wallet(steward['wallet_config'], steward['wallet_credentials'])
    except IndyError as ex:
        if ex.error_code == ErrorCode.WalletAlreadyExistsError:
            pass

    steward['wallet'] = await wallet.open_wallet(steward['wallet_config'], steward['wallet_credentials'])

    print("\"Sovrin Steward\" -> Create and store in Wallet DID from seed")
    steward['did_info'] = json.dumps({'seed': steward['seed']})
    steward['did'], steward['key'] = await did.create_and_store_my_did(steward['wallet'], steward['did_info'])

    print("==============================")
    print("== 3/9: Onboarded Goverment  ==")
    print("== Getting Trust Anchor credentials - Government Onboarding  ==")
    print("------------------------------")

    government = {
        'name': 'Government',
        'wallet_config': json.dumps({'id': 'government_wallet'}),
        'wallet_credentials': json.dumps({'key': 'government_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }
    steward['did_for_government'], steward['key_for_government'], government['did_for_steward'], \
    government['key_for_steward'], _ = await onboarding(steward, government)

    print("==============================")
    print("== Getting Trust Anchor credentials - Government getting Verinym  ==")
    print("------------------------------")

    government['did'] = await get_verinym(steward, steward['did_for_government'], steward['key_for_government'],
                                          government, government['did_for_steward'], government['key_for_steward'])
    
    print("==============================")
    print("== 4/9: Onboarded Faber  ==")
    print("== Getting Trust Anchor credentials - Faber Onboarding  ==")
    print("------------------------------")

    faber = {
        'name': 'Faber',
        'wallet_config': json.dumps({'id': 'faber_wallet'}),
        'wallet_credentials': json.dumps({'key': 'faber_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }
    steward['did_for_faber'], steward['key_for_faber'], faber['did_for_steward'], faber['key_for_steward'], _ = \
        await onboarding(steward, faber)

    print("==============================")
    print("== Getting Trust Anchor credentials - Faber getting Verinym  ==")
    print("------------------------------")

    faber['did'] = \
        await get_verinym(steward, steward['did_for_faber'], steward['key_for_faber'],
                          faber, faber['did_for_steward'], faber['key_for_steward'])
    
    print("Create blob storage")
    faber['tails_writer_config'] = json.dumps({'base_dir': str(os.path.join("/tmp","tails")), 'uri_pattern': ''})
    faber['tails_writer'] = await blob_storage.open_writer('default', faber['tails_writer_config'])
    
    print("==============================")
    print("== 5/9: Onboarded Acme  ==")
    print("== Getting Trust Anchor credentials - Acme Onboarding  ==")
    print("------------------------------")

    acme = {
        'name': 'Acme',
        'wallet_config': json.dumps({'id': 'acme_wallet'}),
        'wallet_credentials': json.dumps({'key': 'acme_wallet_key'}),
        'pool': pool_['handle'],
        'role': 'TRUST_ANCHOR'
    }
    steward['did_for_acme'], steward['key_for_acme'], acme['did_for_steward'], acme['key_for_steward'], _ = \
        await onboarding(steward, acme)
  
    print("==============================")
    print("== Getting Trust Anchor credentials - Acme getting Verinym  ==")
    print("------------------------------")

    acme['did'] = await get_verinym(steward, steward['did_for_acme'], steward['key_for_acme'],
                                    acme, acme['did_for_steward'], acme['key_for_steward'])
  
    print("==============================")
    print("=== 6/9: Schema setup ==")
    print("=== Credential Schemas Setup ==")
    print("------------------------------")

    print("\"Government\" -> Create \"Job-Certificate\" Schema")
    job_certificate = {
        'name': 'Job-Certificate',
        'version': '0.2',
        'attributes': ['first_name', 'last_name', 'salary', 'employee_status', 'experience']
    }
    (government['job_certificate_schema_id'], government['job_certificate_schema']) = \
        await anoncreds.issuer_create_schema(government['did'], job_certificate['name'], job_certificate['version'],
                                             json.dumps(job_certificate['attributes']))
    job_certificate_schema_id = government['job_certificate_schema_id']

    print("\"Government\" -> Send \"Job-Certificate\" Schema to Ledger")
    await send_schema(government['pool'], government['wallet'], government['did'], government['job_certificate_schema'])

    print("\"Government\" -> Create \"PDP\" Schema")
    pdp = {
        'name': 'PDP',
        'version': '1.2',
        'attributes': ['first_name', 'last_name', 'id', 'expiration', 'limitation', 'consent', 'validity_ttl']
    }
    (government['pdp_schema_id'], government['pdp_schema']) = \
        await anoncreds.issuer_create_schema(government['did'], pdp['name'], pdp['version'],
                                             json.dumps(pdp['attributes']))
    pdp_schema_id = government['pdp_schema_id']

    print("\"Government\" -> Send \"PDP\" Schema to Ledger")
    await send_schema(government['pool'], government['wallet'], government['did'], government['pdp_schema'])

    time.sleep(1)  # sleep 1 second before getting schema
  
    print("==============================")
    print("=== 7/9: Faber Consent Receipt Setup ==")
    print("=== Faber Credential Definition Setup ==")
    print("------------------------------")

    print("\"Faber\" -> Get \"PDP\" Schema from Ledger")
    (faber['pdp_schema_id'], faber['pdp_schema']) = \
        await get_schema(faber['pool'], faber['did'], pdp_schema_id)

    print("\"Faber\" -> Create and store in Wallet \"Faber PDP\" Credential Definition")
    pdp_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }
    (faber['pdp_cred_def_id'], faber['pdp_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(faber['wallet'], faber['did'],
                                                               faber['pdp_schema'], pdp_cred_def['tag'],
                                                               pdp_cred_def['type'],
                                                               json.dumps(pdp_cred_def['config']))

    print("\"Faber\" -> Send  \"Faber PDP\" Credential Definition to Ledger")
    await send_cred_def(faber['pool'], faber['wallet'], faber['did'], faber['pdp_cred_def'])

    print("==============================")
    print("=== 8/9: Acme Proof ==")
    print("=== Acme Credential Definition Setup ==")
    print("------------------------------")

    print("\"Acme\" -> Get from Ledger \"Job-Certificate\" Schema")
    (acme['job_certificate_schema_id'], acme['job_certificate_schema']) = \
        await get_schema(acme['pool'], acme['did'], job_certificate_schema_id)

    print("\"Acme\" -> Create and store in Wallet \"Acme Job-Certificate\" Credential Definition")
    job_certificate_cred_def = {
        'tag': 'TAG1',
        'type': 'CL',
        'config': {"support_revocation": False}
    }
    (acme['job_certificate_cred_def_id'], acme['job_certificate_cred_def']) = \
        await anoncreds.issuer_create_and_store_credential_def(acme['wallet'], acme['did'],
                                                               acme['job_certificate_schema'],
                                                               job_certificate_cred_def['tag'],
                                                               job_certificate_cred_def['type'],
                                                               json.dumps(job_certificate_cred_def['config']))

    print("\"Acme\" -> Send \"Acme Job-Certificate\" Credential Definition to Ledger")
    await send_cred_def(acme['pool'], acme['wallet'], acme['did'], acme['job_certificate_cred_def'])
    
    print("==============================")
    print("== 9/9: Alice onboarded  ==")
    print("== Prep - Alice Onboarding  ==")
    print("------------------------------")
    
    alice = {
        'name': 'Alice',
        'wallet_config': json.dumps({'id': 'alice_wallet'}),
        'wallet_credentials': json.dumps({'key': 'alice_wallet_key'}),
        'pool': pool_['handle'],
    }
    faber['did_for_alice'], faber['key_for_alice'], alice['did_for_faber'], alice['key_for_faber'], \
    faber['alice_connection_response'] = await onboarding(faber, alice)
  
    print("Getting started -> done")
    
    print("*********************************************")
    print("*********************************************")
    print("*** Creating Consent Receipt Certificatem ***")
    print("*********************************************")
    print("*********************************************")

    print("==============================")
    print("=== Getting PDP with Faber ==")
    print("==============================")
    print("== Getting PDP with Faber - Onboarding ==")
    print("------------------------------")

    faber['did_for_alice'], faber['key_for_alice'], alice['did_for_faber'], alice['key_for_faber'], \
    faber['alice_connection_response'] = await onboarding(faber, alice)

    print("==============================")
    print("== Getting PDP with Faber - Getting PDP Credential ==")
    print("------------------------------")

    print("\"Faber\" -> Create \"PDP\" Credential Offer for Alice")
    faber['pdp_cred_offer'] = \
        await anoncreds.issuer_create_credential_offer(faber['wallet'], faber['pdp_cred_def_id'])

    print("\"Faber\" -> Get key for Alice did")
    faber['alic_key_for_faber'] = \
        await did.key_for_did(faber['pool'], faber['wallet'], faber['alice_connection_response']['did'])

    print("\"Faber\" -> Authcrypt \"PDP\" Credential Offer for Alice")
    faber['authcrypted_pdp_cred_offer'] = \
        await crypto.auth_crypt(faber['wallet'], faber['key_for_alice'], faber['alic_key_for_faber'],
                                faber['pdp_cred_offer'].encode('utf-8'))

    print("\"Faber\" -> Send authcrypted \"PDP\" Credential Offer to Alice")
    alice['authcrypted_pdp_cred_offer'] = faber['authcrypted_pdp_cred_offer']
    print("\"Alice\" -> Authdecrypted \"PDP\" Credential Offer from Faber")
    alice['faber_key_for_alice'], alice['pdp_cred_offer'], authdecrypted_pdp_cred_offer = \
        await auth_decrypt(alice['wallet'], alice['key_for_faber'], alice['authcrypted_pdp_cred_offer'])
    alice['pdp_schema_id'] = authdecrypted_pdp_cred_offer['schema_id']
    alice['pdp_cred_def_id'] = authdecrypted_pdp_cred_offer['cred_def_id']


    # privacy agreement
    form_item_layout = Layout(
        display='flex',
        flex_flow='row',
        justify_content='space-between'
    )
    
    form_items = [
        Box([HTML(value= '''
    <H2>Privay Policy</H2>
    <U>Purpose</U><BR>
    Private Data is collected to improve services. The data is used for purpose a, b and c.<BR>
    <U>Data collected</U><BR>
    Data is collected while using the service which expires on 2020-07-01. After this period if 
    service is not renewed then no more data is collected.<BR>
    <U>Data limitation</U><BR>
    Data is only kept for a duration of 1 year.<BR>
    <U>Data sharing</U><BR>
    Data may be shared with your consent with Acme to provided additional service of 
    recommendation on study plan in order to secure a job after college.
                  ''')], layout=form_item_layout),
        Box([HTML(value='Once you have read the privacy policy indicate if you agree with the conditions of using of your \
                private data.')], layout=form_item_layout)
    ]

    form = Box(form_items, layout=Layout(
        display='flex',
        flex_flow='column',
        border='solid 2px',
        align_items='stretch',
        width='100%'
    ))

    out_cert = Output()
    display(VBox([form,out_cert]))

    print("\"Alice\" -> Create and store \"Alice\" Master Secret in Wallet")
    alice['master_secret_id'] = await anoncreds.prover_create_master_secret(alice['wallet'], None)

    print("\"Alice\" -> Get \"Faber PDP\" Credential Definition from Ledger")
    (alice['faber_pdp_cred_def_id'], alice['faber_pdp_cred_def']) = \
        await get_cred_def(alice['pool'], alice['did_for_faber'], alice['pdp_cred_def_id'])

    print("\"Alice\" -> Create \"PDP\" Credential Request for Faber")
    (alice['pdp_cred_request'], alice['pdp_cred_request_metadata']) = \
        await anoncreds.prover_create_credential_req(alice['wallet'], alice['did_for_faber'],
                                                     alice['pdp_cred_offer'], alice['faber_pdp_cred_def'],
                                                     alice['master_secret_id'])

    print("\"Alice\" -> Authcrypt \"PDP\" Credential Request for Faber")
    alice['authcrypted_pdp_cred_request'] = \
        await crypto.auth_crypt(alice['wallet'], alice['key_for_faber'], alice['faber_key_for_alice'],
                                alice['pdp_cred_request'].encode('utf-8'))

    #TODO: values come from entry overlay
    
    print("\"Alice\" -> Send authcrypted \"PDP\" Credential Request to Faber")
    alice['pdp_cred_values'] = json.dumps({
        "first_name": {"raw": "Alice", "encoded": "1139481716457488690172217916278103335"},
        "last_name": {"raw": "Garcia", "encoded": "5321642780241790123587902456789123452"},
        "id": {"raw": "123-45-6789", "encoded": "3124141231422543541"},
        "expiration": {"raw": "2020-07-01", "encoded": "20200701"},
        "limitation": {"raw": "360", "encoded": "360"},
        "consent": {"raw": "1", "encoded": "1"},
        "validity_ttl": {"raw": "3", "encoded": "3"}
    })
        
    faber['authcrypted_pdp_cred_request'] = alice['authcrypted_pdp_cred_request']
    faber['alice_pdp_cred_values'] = alice['pdp_cred_values']

    print("\"Faber\" -> Authdecrypt \"PDP\" Credential Request from Alice")
    faber['alice_key_for_faber'], faber['pdp_cred_request'], _ = \
        await auth_decrypt(faber['wallet'], faber['key_for_alice'], faber['authcrypted_pdp_cred_request'])
    
    #TODO: verify that default values are not modified/hacked

    print("\"Faber\" -> Create \"PDP\" Credential for Alice")

    #  Issuer Opens Tails reader
    faber['blob_storage_reader_cfg_handle'] = await blob_storage.open_reader('default', faber['tails_writer_config'])
    
    # pdp_cred is cred_json
    faber['pdp_cred'], faber['cred_revoc_id'], faber['revoc_reg_delta_json'] = \
        await anoncreds.issuer_create_credential(faber['wallet'], faber['pdp_cred_offer'],
                                                 faber['pdp_cred_request'],
                                                 faber['alice_pdp_cred_values'], None, None)


    print("\"Faber\" -> Authcrypt \"PDP\" Credential for Alice")
    faber['authcrypted_pdp_cred'] = \
        await crypto.auth_crypt(faber['wallet'], faber['key_for_alice'], faber['alice_key_for_faber'],
                                faber['pdp_cred'].encode('utf-8'))

    print("\"Faber\" -> Send authcrypted \"PDP\" Credential to Alice")
    alice['authcrypted_pdp_cred'] = faber['authcrypted_pdp_cred']

    print("\"Alice\" -> Authdecrypted \"PDP\" Credential from Faber")
    _, alice['pdp_cred'], _ = \
        await auth_decrypt(alice['wallet'], alice['key_for_faber'], alice['authcrypted_pdp_cred'])

    #TODO: Alice should verify that same values as she sends it
    print("\"Alice\" -> Store \"PDP\" Credential from Faber")
    _, alice['pdp_cred_def'] = await get_cred_def(alice['pool'], alice['did_for_faber'],
                                                         alice['pdp_cred_def_id'])

    await anoncreds.prover_store_credential(alice['wallet'], None, alice['pdp_cred_request_metadata'],
                                            alice['pdp_cred'], alice['pdp_cred_def'], None)
    
    print('Consent Receipt Certificate')
    print(alice['pdp_cred'])
        
    print("*********************************************")
    print("*********************************************")
    print("*** Performing Proof (without PII)        ***")
    print("*********************************************")

    print("==============================")
    print("=== Apply for the job with Acme ==")
    print("==============================")
    print("== Apply for the job with Acme - Onboarding ==")
    print("------------------------------")

    acme['did_for_alice'], acme['key_for_alice'], alice['did_for_acme'], alice['key_for_acme'], \
    acme['alice_connection_response'] = await onboarding(acme, alice)

    print("==============================")
    print("== Apply for the job with Acme - PDP proving ==")
    print("------------------------------")

    print("\"Acme\" -> Create \"PDP\" Proof Request")
    acme['pdp_proof_request'] = json.dumps({
        'nonce': '1432422343242122312411212',
        'name': 'PDP',
        'version': '0.1',
        'requested_attributes': {
            'attr2_referent': {
                'name': 'consent',
                'restrictions': [{'cred_def_id': faber['pdp_cred_def_id']}]
            }            
,
            'attr3_referent': {
                'name': 'validity_ttl',
                'restrictions': [{'cred_def_id': faber['pdp_cred_def_id']}]
            }            
        },
        'requested_predicates': {}
    })
    
    print("\"Acme\" -> Get key for Alice did")
    acme['alice_key_for_acme'] = \
        await did.key_for_did(acme['pool'], acme['wallet'], acme['alice_connection_response']['did'])

    print("\"Acme\" -> Authcrypt \"PDP\" Proof Request for Alice")
    acme['authcrypted_pdp_proof_request'] = \
        await crypto.auth_crypt(acme['wallet'], acme['key_for_alice'], acme['alice_key_for_acme'],
                                acme['pdp_proof_request'].encode('utf-8'))

    print("\"Acme\" -> Send authcrypted \"PDP\" Proof Request to Alice")
    alice['authcrypted_pdp_proof_request'] = acme['authcrypted_pdp_proof_request']

    print("\"Alice\" -> Authdecrypt \"PDP\" Proof Request from Acme")
    alice['acme_key_for_alice'], alice['pdp_proof_request'], _ = \
        await auth_decrypt(alice['wallet'], alice['key_for_acme'], alice['authcrypted_pdp_proof_request'])

    print("\"Alice\" -> Get credentials for \"PDP\" Proof Request")

    search_for_pdp_proof_request = \
        await anoncreds.prover_search_credentials_for_proof_req(alice['wallet'],
                                                                alice['pdp_proof_request'], None)

    cred_for_attr2 = await get_credential_for_referent(search_for_pdp_proof_request, 'attr2_referent')
    cred_for_attr3 = await get_credential_for_referent(search_for_pdp_proof_request, 'attr3_referent')

    await anoncreds.prover_close_credentials_search_for_proof_req(search_for_pdp_proof_request)

    alice['creds_for_pdp_proof'] = {cred_for_attr2['referent']: cred_for_attr2,
                                    cred_for_attr3['referent']: cred_for_attr3}

    alice['schemas'], alice['cred_defs'], alice['revoc_states'] = \
        await prover_get_entities_from_ledger(alice['pool'], alice['did_for_acme'],
                                              alice['creds_for_pdp_proof'], alice['name'])

    print("\"Alice\" -> Create \"PDP\" Proof")
    alice['pdp_requested_creds'] = json.dumps({
        'self_attested_attributes': {},
        'requested_attributes': {
            'attr2_referent': {'cred_id': cred_for_attr2['referent'], 'revealed': True},
            'attr3_referent': {'cred_id': cred_for_attr3['referent'], 'revealed': True}
        },
        'requested_predicates': {}
    })

    alice['pdp_proof'] = \
        await anoncreds.prover_create_proof(alice['wallet'], alice['pdp_proof_request'],
                                            alice['pdp_requested_creds'], alice['master_secret_id'],
                                            alice['schemas'], alice['cred_defs'], alice['revoc_states'])

    print("\"Alice\" -> Authcrypt \"PDP\" Proof for Acme")
    alice['authcrypted_pdp_proof'] = \
        await crypto.auth_crypt(alice['wallet'], alice['key_for_acme'], alice['acme_key_for_alice'],
                                alice['pdp_proof'].encode('utf-8'))

    print("\"Alice\" -> Send authcrypted \"PDP\" Proof to Acme")
    acme['authcrypted_pdp_proof'] = alice['authcrypted_pdp_proof']
    
    print("\"Acme\" -> Authdecrypted \"PDP\" Proof from Alice")
    _, acme['pdp_proof'], decrypted_pdp_proof = \
        await auth_decrypt(acme['wallet'], acme['key_for_alice'], acme['authcrypted_pdp_proof'])

    acme['schemas'], acme['cred_defs'], acme['revoc_ref_defs'], acme['revoc_regs'] = \
        await verifier_get_entities_from_ledger(acme['pool'], acme['did'],
                                                decrypted_pdp_proof['identifiers'], acme['name'])

    print("\"Acme\" -> Verify \"PDP\" Proof from Alice")
    try:
        assert '1' == \
               decrypted_pdp_proof['requested_proof']['revealed_attrs']['attr2_referent']['raw']
        assert2 = ' confirmed'
    except:
        assert2 = ' failed'
        
    print("\"Acme\" -> Verify \"PDP\" Proof from Alice")
    try:
        assert '3' == \
               decrypted_pdp_proof['requested_proof']['revealed_attrs']['attr3_referent']['raw']
        assert3 = ' confirmed'
    except:
        assert3 = ' failed'
    try:
        assert await anoncreds.verifier_verify_proof(acme['pdp_proof_request'], acme['pdp_proof'],
                                                     acme['schemas'], acme['cred_defs'], acme['revoc_ref_defs'],
                                                     acme['revoc_regs'])
        proof1 = 'Proof confirmed'
    except AssertionError as e:
        proof1 = 'Proof failed'
        
    print('PDP Proof Request')
    print(json.loads(acme['pdp_proof_request']))
    print('Attribute '+json.loads(acme['pdp_proof_request'])['requested_attributes']['attr2_referent']['name'] + assert2)
    print('Attribute '+json.loads(acme['pdp_proof_request'])['requested_attributes']['attr3_referent']['name'] + assert3)
    print(proof1)
        
    print("*********************************************")
    print("*********************************************")
    print("*** Close ledger                          ***")
    print("*********************************************")

    print(" \"Sovrin Steward\" -> Close and Delete wallet")
    await wallet.close_wallet(steward['wallet'])
    await wallet.delete_wallet(steward['wallet_config'], steward['wallet_credentials'])

    print("\"Government\" -> Close and Delete wallet")
    await wallet.close_wallet(government['wallet'])
    await wallet.delete_wallet(government['wallet_config'], government['wallet_credentials'])
    
    try:
        print("\"Faber\" -> Close and Delete wallet")
        await wallet.close_wallet(faber['wallet'])
        await wallet.delete_wallet(faber['wallet_config'], faber['wallet_credentials'])
    except:
        pass
    
    try:
        print("\"Acme\" -> Close and Delete wallet")
        await wallet.close_wallet(acme['wallet'])
        await wallet.delete_wallet(acme['wallet_config'], acme['wallet_credentials'])
    except:
        pass
    
    try:
        print("\"Alice\" -> Close and Delete wallet")
        await wallet.close_wallet(alice['wallet'])
        await wallet.delete_wallet(alice['wallet_config'], alice['wallet_credentials'])
    except:
        pass
        
    print("Close and Delete pool")
    await pool.close_pool_ledger(pool_['handle'])
    await pool.delete_pool_ledger_config(pool_['name'])
    
if __name__ == '__main__':
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(run())
    time.sleep(1)  # FIXME waiting for libindy thread complete


Getting started -> started
=== 1/9: Open Ledger  ==
Open Pool Ledger: pool1
== 2/9: Setup Steward  ==
=== Getting Trust Anchor credentials for Faber, Acme, Thrift and Government  ==
------------------------------
"Sovrin Steward" -> Create wallet
"Sovrin Steward" -> Create and store in Wallet DID from seed
== 3/9: Onboarded Goverment  ==
== Getting Trust Anchor credentials - Government Onboarding  ==
------------------------------
"Sovrin Steward" -> Create and store in Wallet "Sovrin Steward Government" DID
"Sovrin Steward" -> Send Nym to Ledger for "Sovrin Steward Government" DID
"Sovrin Steward" -> Send connection request to Government with "Sovrin Steward Government" DID and nonce
"Government" -> Create wallet
"Government" -> Create and store in Wallet "Government Sovrin Steward" DID
"Government" -> Get key for did from "Sovrin Steward" connection request
"Government" -> Anoncrypt connection response for "Sovrin Steward" with "Government Sovrin Steward" DID, verkey and nonce
"Gover

"Alice" -> Create and store "Alice" Master Secret in Wallet
"Alice" -> Get "Faber PDP" Credential Definition from Ledger
"Alice" -> Create "PDP" Credential Request for Faber
"Alice" -> Authcrypt "PDP" Credential Request for Faber
"Alice" -> Send authcrypted "PDP" Credential Request to Faber
"Faber" -> Authdecrypt "PDP" Credential Request from Alice
"Faber" -> Create "PDP" Credential for Alice
"Faber" -> Authcrypt "PDP" Credential for Alice
"Faber" -> Send authcrypted "PDP" Credential to Alice
"Alice" -> Authdecrypted "PDP" Credential from Faber
"Alice" -> Store "PDP" Credential from Faber
Consent Receipt Certificate
{"schema_id":"QFhKsBzE8ZVThtdEc4swwf:2:PDP:1.2","cred_def_id":"G2BcB9MJCuGxGAKeYX62u8:3:CL:541:TAG1","rev_reg_id":null,"values":{"expiration":{"raw":"2020-07-01","encoded":"20200701"},"id":{"raw":"123-45-6789","encoded":"3124141231422543541"},"first_name":{"raw":"Alice","encoded":"1139481716457488690172217916278103335"},"limitation":{"raw":"360","encoded":"360"},"last_name"